## Potrzebne importy

In [134]:
import numpy as np
import pandas as pd
import csv
import datetime
import os

# **Dane o zużyciu energii**

## Funkcja przygotowująca dane o zużyciu energii z PSE

In [135]:
def prepare_energy_data(filename):
    df = pd.read_csv(filename,sep = ';')
    df['Data']= pd.to_datetime(df['Data'],format='%Y%m%d')
    
    if(df['Dobowa prognoza zapotrzebowania KSE'].dtype != int):
        df['Dobowa prognoza zapotrzebowania KSE'] = df['Dobowa prognoza zapotrzebowania KSE'].str.replace(',', '.')
        df['Dobowa prognoza zapotrzebowania KSE'] = pd.to_numeric(df['Dobowa prognoza zapotrzebowania KSE'],errors='coerce')
        
    df['Rzeczywiste zapotrzebowanie KSE'] = df['Rzeczywiste zapotrzebowanie KSE'].str.replace(',', '.')
    df['Rzeczywiste zapotrzebowanie KSE'] = pd.to_numeric(df['Rzeczywiste zapotrzebowanie KSE'],errors='coerce')
    
    df_daily = df.groupby("Data",sort=False)[['Dobowa prognoza zapotrzebowania KSE','Rzeczywiste zapotrzebowanie KSE']].mean().reset_index()

    return df,df_daily

## Wykonanie funkcji na wszystkich danych i połączenie ich w nowy dataframe

In [136]:
hourly = pd.DataFrame(columns = ['Data', 'Godz.','Dobowa prognoza zapotrzebowania KSE', 'Rzeczywiste zapotrzebowanie KSE'])
daily = pd.DataFrame(columns = ['Data','Dobowa prognoza zapotrzebowania KSE', 'Rzeczywiste zapotrzebowanie KSE'])

cwd = os.getcwd()
directory = os.path.join(cwd,"energy_consumption_data")

for d in os.listdir(directory):
    path = os.path.join(directory,d)
    if os.path.isfile(path) and path.endswith(".csv"):
        df, df_daily = prepare_energy_data(path)
        hourly = pd.concat([hourly,df])
        daily = pd.concat([daily,df_daily])
            

## Podział danych do dwóch plików CSV: energy_daily.csv i energy_hourly.csv

In [137]:
hourly = hourly.sort_values(by="Data")
hourly = hourly.round(2)

daily = daily.sort_values(by="Data")
daily = daily.round(2)


hourly.to_csv('energy_hourly.csv', index=False)
daily.to_csv('energy_daily.csv', index=False)

<br>

# **Dane pogodowe**

## Funkcja przygotowująca dane o pogodzie

In [138]:
def prepare_weather_data(filename):
    with open(filename, 'r',encoding='utf-8', errors='ignore') as file:
        data = file.read().replace('"', '')
    with open(filename,'w',encoding='utf-8', errors='ignore') as file:
        file.write(data)
        
    # wczytanie danych i rename column
    df = pd.read_csv(filename,sep = ',',header = None)
    df = df.rename(columns={0: 'id', 1: 'station',2: 'year',3: 'month', 4: 'day', 5: 'Temperatura',
                   6: 'del1',7: 'Wilogtność powietrza',8: 'del2',9: 'Prędkość wiatru',10: 'del3',11: 'Zachmurzenie',12: 'del4'})

    # połączenie daty
    cols=["year","month","day"]
    df['Data'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
    
    # usunięcie niepotrzebnych kolumn
    df = df.drop(columns=['id','station', 'del1','del2','del3','del4','year','day','month']) 
    
    # czyszczenie i grupowanie danych
#     df["temperature"] = df["temperature"].replace(0.0,np.nan)
    df["Zachmurzenie"] = df["Zachmurzenie"].replace(0.0,np.nan)
#     df["wind_speed"] = df["wind_speed"].replace(0.0,np.nan)
#     df["cloudiness"] = df["cloudiness"].replace(0.0,np.nan)
    df = df.groupby("Data",sort=False)[['Data', 'Temperatura','Wilogtność powietrza', 'Prędkość wiatru', 'Zachmurzenie']].mean().reset_index()
    
    return df

## Wykonanie funkcji na wszystkich danych i połączenie ich w nowy dataframe

In [139]:
main = pd.DataFrame(columns = ['Data', 'Temperatura','Wilogtność powietrza', 'Prędkość wiatru', 'Zachmurzenie'])

# dane pochodzą z: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/

cwd = os.getcwd()
directory = os.path.join(cwd,"weather_data")

for d in os.listdir(directory):
    path = os.path.join(directory,d)
    if os.path.isdir(path) and not path.startswith('.') and  not path.startswith('..'):
        for filename in os.listdir(os.path.join(directory,path)):
            if filename.endswith(".csv") and filename.startswith("k_d_t_") and filename != 'out.csv':
                df = prepare_weather_data(os.path.join(path,filename))
                main = pd.concat([main,df])

## Dodatkowe czyszczenie danych i zapis do pliku weather.csv

In [140]:
main['Data'] = pd.to_datetime(main["Data"])
main = main.sort_values(by="Data")
main = main.round(2)

start_date = '2015-01-01'
end_date = '2021-01-31'

mask = (main['Data'] >= start_date) & (main['Data'] <= end_date)

main = main.loc[mask]

main.to_csv('weather.csv', index=False)

# Stworzenie dodatkowego pliku z dziennymi danymi

In [141]:
main['Dobowa prognoza zapotrzebowania KSE'] = daily['Dobowa prognoza zapotrzebowania KSE'].values
main['Rzeczywiste zapotrzebowanie KSE'] = daily['Rzeczywiste zapotrzebowanie KSE'].values

In [142]:
main.head()

,Data,Temperatura,Wilogtność powietrza,Prędkość wiatru,Zachmurzenie,Dobowa prognoza zapotrzebowania KSE,Rzeczywiste zapotrzebowanie KSE
0,2015-01-01,-0.77,31.80,1.34,7.57,NaN,15098.40
1,2015-01-02,1.27,29.94,2.47,6.83,17822.96,17869.20
2,2015-01-03,1.87,28.00,2.80,6.95,17639.33,17684.92
3,2015-01-04,0.42,28.89,2.79,6.27,16235.42,16453.76
4,2015-01-05,-1.03,29.75,1.91,6.05,18892.71,18856.44


In [143]:
main.to_csv('daily_weather_and_energy_consumption.csv', index=False)